# Isopy Tutorial 1 - Arrays and Reference Values

In this tutorial we provide brief introduction to isopy arrays, reference values and ways to use them. For a thorough description of the functions and methods available see the 

Before we start we have to import the isopy library and the numpy libraries,

In [1]:
import isopy
import numpy as np

## Array Introduction

---
An isopy array is subclass of a numpy array with a number of special features aimed at working with geochemical data. The data is stored in rows and columns, each column has a key with a certain flavour describing the type of data it represents. There are 6 key string flavours, in order of priority, they are:

- `mass` consists of a mass number. Only accepts positive integers. E.g `"102"` or $102$
- `element` consists of an element symbol. E.g `"Pd"` or $\mathrm{Pd}$
- `isotope` consists of a mass number followed by a element symbol. E.g `"102Pd"` or ${}^{102}\mathrm{Pd}$
- `molecule` consists of multiple element and/or isotope keys. E.g. `"[H2O]"` or $\mathrm{H}_{2}\mathrm{O}$
- `ratio` consists of a numerator and a denominator key string. E.g. `"108Pd/105Pd"` or $\cfrac{{}^{108}\mathrm{Pd}}{{}^{105}\mathrm{Pd}}$
- `general` accepts any string. E.g. `"whatever"` or $\mathrm{whatever}$

The highest priority flavour, that the input string is compatible with, is used for the array. Each key string type adheres to a strict format but it is flexible with the formatting of the input string. For example the `102pd`, `Pd102` and `palladium-102` will all yield the isotope key string `102Pd`. You can read more about key strings in the {ref}`here<Key Strings>`.

You can create arrays using the {func}`isopy.array` function, 

In [40]:
data = {'ru': [1, 2], 'pd': [3, 4], 'cd': [5, 6]}
a = isopy.array(data); a # column keys inferred from the dictionary

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,1.00000,3.00000,5.00000
1,2.00000,4.00000,6.00000


## Indexing

---
You can index arrays using either the row number or the column key. You can also use the {meth}`IsopyArray.get()<isopy.core.IsopyArray.get>` method to fetch the column which will return a default value if a column does not exits in the array.

In [3]:
a['pd'] # key string automatically formatted

array([3., 4.])

In [4]:
a[1]

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
None,2.00000,4.00000,6.00000


In [5]:
a.get('ge'), a.get('ge', 1)

(array([nan, nan]), array([1., 1.]))

## Attributes & Methods

---
The `nrows`, `ncols` and the `keys` attributes will return the number of row, number of columns and the keys for each column in the array respectively. For a full list of attributes and methods see the {class}`IsopyArray<isopy.core.IsopyArray>` documentation.

In [6]:
a.nrows, a.ncols, a.keys

(2, 3, IsopyKeyList('Ru', 'Pd', 'Cd', flavour='element'))

---
The {meth}`IsopyArray.ratio()<isopy.core.IsopyArray.ratio>` method will divide the value in each column with the value of the given column creating a new array with ratio column keys. The {meth}`IsopyArray.deratio()<isopy.core.IsopyArray.deratio>` method will reverse it.

In [7]:
b = a.ratio('pd'); b

(row),$\cfrac{\mathrm{Ru}}{\mathrm{Pd}}$ (f8),$\cfrac{\mathrm{Cd}}{\mathrm{Pd}}$ (f8)
0,0.33333,1.66667
1,0.50000,1.50000


In [8]:
b.deratio()

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,0.33333,1.00000,1.66667
1,0.50000,1.00000,1.50000


---
The {meth}`IsopyArray.normalise()<isopy.core.IsopyArray.normalise>` method will by default normalise the array such that the sum of each row is equal to 1. You can also specify the normalisation value for a specific set of keys. Note that the method return a new array.

In [9]:
a.normalise()

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,0.11111,0.33333,0.55556
1,0.16667,0.33333,0.50000


In [10]:
a.normalise([10], 'pd') # Renormalises each row so the Pd columns is equal to 10

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,3.33333,10.00000,16.66667
1,5.00000,10.00000,15.00000


---
Finally, there are several functions for turning array into other data types. For example the {meth}`IsopyArray.to_dict()<isopy.core.IsopyArray.to_dict>` method will convert the array to a dictionary, e.g.

In [11]:
a.to_dict()

{'Ru': [1.0, 2.0], 'Pd': [3.0, 4.0], 'Cd': [5.0, 6.0]}

## Creating Arrays

---
Arrays can be created with the {func}`isopy.array` function. Each array must consist of data and a set of keys, if it cannot be inferred from the data. 

In [12]:
isopy.array([[1, 3, 5], [2, 4, 6]],'ru pd cd')
isopy.array({'ru': [1, 2], 'pd': [3, 4], 'cd': [5, 6]})
isopy.array(ru = [1, 2], pd=[3, 4], cd=[5, 6])

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,1.00000,3.00000,5.00000
1,2.00000,4.00000,6.00000


You can also create arrays from existing isopy arrays, structured numpy arrays and pandas dataframes. If the input is an isopy array {func}`isopy.array` will return a copy of the array while {func}`isopy.asarray` will return a reference to it, assuming it satisfies any other arguments given.

In [13]:
a = isopy.array([[1, 3, 5], [2, 4, 6]],'ru pd cd')
b = isopy.array(a)
c = isopy.asarray(a)
a is b, a is c

(False, True)

There are functions to create arrays filled with 0's, 1's or random numbers. See the {ref}`Arrays` for more details. For example an array filled with a normal distribution of random values can be created using the {func}`isopy.random` function

In [14]:
isopy.random(10, keys = 'ru pd cd')

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,0.94193,-0.84010,-1.46872
1,0.84349,0.19054,-0.92886
2,-0.68974,-1.30829,0.61475
3,1.32635,0.58710,0.21810
4,-0.40937,0.32379,1.90051
5,1.31348,-1.00570,0.00971
6,0.59463,-0.27932,-0.33032
7,0.34017,-1.29789,0.12494
8,-0.73290,-0.63486,-0.86405
9,3.10324,-1.40903,0.66490


---
Arrays can be zero or one dimensional depending on the input. The `ndim` attribute can be used to check the number of dimensions of the array. If the array is 0-dimensional `nrows` will be have a value of `-1`.

In [15]:
isopy.array({'ru': [1], 'pd': [3], 'cd': [5]})

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,1.00000,3.00000,5.00000


In [16]:
isopy.array({'ru': 1, 'pd': 3, 'cd': 5})

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
None,1.00000,3.00000,5.00000


You can use the `ndim` keyword to specify the number of dimensions of the array. If `ndim=-1` then the returned array will be 0-dimensional if possible otherwise 1-dimensional.

In [17]:
isopy.array({'ru': [1], 'pd': [3], 'cd': [5]}, ndim=-1)

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
None,1.00000,3.00000,5.00000


---
By default the data type for python objects will be `float64` if possible and otherwise whatever numpy suggests. The datatype is given in abbreviated in parenthesis next to the column key in the repr of the array. The data type of each column can also be found from the `datatypes` attribute.

In [18]:
isopy.array({'ru': [1, 2], 'pd': [3, 4], 'cd': [5, 6]}).datatypes

(dtype('float64'), dtype('float64'), dtype('float64'))

If you create an array from a numpy array them it will inherit the data type of the numpy array.

In [19]:
isopy.array({'ru': np.array([1, 2]), # dtype = int64
             'pd': np.array([3, 4], dtype=np.float64), 
             'cd': np.array([5, 6], dtype=str)}).datatypes

(dtype('int64'), dtype('float64'), dtype('<U1'))

You can also specify the data type for each column using the `dtype` keyword. You can either pass a single datatype, that will be used for each column, or a **list** of datatypes, one for each column.

In [20]:
isopy.array({'ru': [1, 2], 'pd': [3, 4], 'cd': [5, 6]}, dtype=np.int64).datatypes

(dtype('int64'), dtype('int64'), dtype('int64'))

---
When creating an array the highest priority key string flavour, that the string is compatible with, is used for the column key. Optionally, you can specify the flavour of the column keys using the `flavour` key word, multiple flavours should be separated with a `|`. 

In [21]:
isopy.array({'ru': [1, 2], 'pd': [3, 4], 'cd': [5, 6]}, flavour='general')

(row),$\mathrm{ru}$ (f8),$\mathrm{pd}$ (f8),$\mathrm{cd}$ (f8)
0,1.00000,3.00000,5.00000
1,2.00000,4.00000,6.00000


## Array Functions

---
We loosely define any function that accepts one or more values, performs a mathematical calculation and returns a new value as an array function. The simplest ones being the `+`, `-`, `*`, `/` and `**` operators. These operators are fully supported by isopy arrays.

In [22]:
a = isopy.array({'ru': [1, 2], 'pd': [3, 4], 'cd': [5, 6]})
a * 2 # Multiplies each column by 2

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,2.00000,6.00000,10.00000
1,4.00000,8.00000,12.00000


In [23]:
[1, 10] - a

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
0,0.00000,-2.00000,-4.00000
1,8.00000,6.00000,4.00000


If combining two isopy arrays then the result will contain the combined keys of both arrays. A default value, `nan`, will be used for missing columns.

In [24]:
b = isopy.array([11, 12, 13], keys='ru ag cd')
a + b

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Ag}$ (f8),$\mathrm{Cd}$ (f8)
0,12.00000,nan,nan,18.00000
1,13.00000,nan,nan,19.00000


You can set a temporary default value using the {meth}`IsopyArray.default()<isopy.core.IsopyArray.default>` method when performing calculations. 

In [25]:
a.default(10) + b 

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Ag}$ (f8),$\mathrm{Cd}$ (f8)
0,12.00000,nan,22.00000,18.00000
1,13.00000,nan,22.00000,19.00000


---
Isopy arrays natively support a wide range of numpy functions. By default the result will be computed on each column and the result compiled into a new array. However, for functions that support it, you can set `axis` to `None` to compute the result over the entire array or `1`  to compute the result for each row.

All the numpy functions that have been tested with isopy arrays are listed {ref}`here<Numpy Functions>`.

In [26]:
np.sum(a)

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
None,3.00000,7.00000,11.00000


In [27]:
np.sum(a, axis=None), np.sum(a, axis=1)

(21.0, array([ 9., 12.]))

All the numpy function that have been tested with isopy arrays are also avaliable from the `isopy` namespace. These functions have been enhanced so that they accept a `keys` argument where you can specify which keys the return array should have.

In [28]:
isopy.sum(a, keys='ru cd')

(row),$\mathrm{Ru}$ (f8),$\mathrm{Cd}$ (f8)
None,3.00000,11.00000


---
In addition isopy also comes with a set of custom array functions documented {ref}`here<Isopy Functions>`. For example the {func}`isopy.sd` and {func}`isopy.se` functions compute the standard deviation and standard error of the sample (1 degree of freedom), respectively.

In [29]:
c = isopy.random(100, keys='ru pd cd')
isopy.se(c) # The standard deviation of the distribution is 1 by default

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
None,0.08599,0.09690,0.09187


These functions also allow you to specify the `zscore` or the confidence interval, `ci`, of a T distribution. E.g.

In [30]:
isopy.se(c, ci=0.95) # 95 % confidence interval

(row),$\mathrm{Ru}$ (f8),$\mathrm{Pd}$ (f8),$\mathrm{Cd}$ (f8)
None,0.17063,0.19228,0.18230


## Reference Values

---
Reference values are constants or literature values, such as the mass of each isotope, that are often used in calculations. There are, where appropriate, stored in a custom dictionary, with a few special features, as scalar values.

Isopy comes with several reference values, available in the `refval` namespace documented {ref}`here<refvaldoc>`. For convinience the repr of the `isopy.refval` object return a list of the avaliable reference values and a short description.

In [31]:
isopy.refval

**isopy.refval.mass** contains the following reference values
* **isotopes** - Dictionary containing all naturally occuring isotopes with a given mass number.

**isopy.refval.element** contains the following reference values
* **isotopes** - Dictionary containing all naturally occurring isotopes for each element symbol.
* **all_symbols** - A tuple of all the element symbols.
* **symbol_name** - Dictionary containing the full element name mapped to the element symbol.
* **name_symbol** - Dictionary containing the element symbol mapped to the full element name.
* **atomic_weight** - Dictionary containing the atomic weight for each element symbol.
* **atomic_number** - Dictionary containing the atomic number for each element symbol.
* **initial_solar_system_abundance_L09** - Dictionary containing the element abundance of the initial solar system composition (normalized to N(Si) = 10^6 atoms) from Lodders et al. (2019).

**isopy.refval.isotope** contains the following reference values
* **mass** - Dictionary containing the default mass of each isotope.
* **fraction** - Dictionary containing the default fraction of each isotope.
* **mass_W17** - Dictionary containing isotope mass of each isotope from Wang et al. (2016).
* **mass_AME20** - Dictionary containing isotope mass of each isotope from the 2020 Atomic Mass Evaluation.
* **mass_number** - Dictionary containing mass number of each isotope.
* **best_measurement_fraction_M16** - Dictionary containing the isotope fraction from the best avaliable measurement from Meija et al. (2016).
* **initial_solar_system_fraction_L09** - Dictionary containing the isotope fraction of the inital solar system composition from Lodders et al. (2019).
* **initial_solar_system_abundance_L09** - Dictionary containing the isotope abundance of the inital solar system composition (normalized to N(Si) = 10^6 atoms) from Lodders et al. (2019).
* **initial_solar_system_abundance_L09b** - Dictionary containing the isotope abundance calcualted from the elemental abundance and the isotope fraction.
* **present_solar_system_fraction_AG89** - Dictionary containing the isotope fraction of the present solar system composition from Anders & Grevesse 1989.
* **initial_solar_system_abundance_AG89** - Dictionary containing the isotope abundance of the initial solar system abundance from Anders & Grevesse 1989.
* **present_solar_system_abundance_AG89** - Dictionary containing the isotope abundance of the present solar system abundance from Anders & Grevesse 1989.
* **sprocess_fraction_B11** - Dictionary containing the estimated s-process fraction of each isotope from Bisterzo et al. (2011).

---
There are two ways of retrieving the reference values that come with isopy. Directly though `refval` object or by passing the name to the {func}`isopy.asrefval` function. **Note** that the latter only works for reference values which contain scalars.

In [32]:
isopy.refval.isotope.fraction # or isopy.asrefval('isotope.fraction')

(row),${}^{1}\mathrm{H}$ (f8),${}^{2}\mathrm{H}$ (f8),${}^{3}\mathrm{He}$ (f8),${}^{4}\mathrm{He}$ (f8),${}^{6}\mathrm{Li}$ (f8),${}^{7}\mathrm{Li}$ (f8),${}^{9}\mathrm{Be}$ (f8),${}^{10}\mathrm{B}$ (f8),${}^{11}\mathrm{B}$ (f8),${}^{12}\mathrm{C}$ (f8),${}^{13}\mathrm{C}$ (f8),${}^{14}\mathrm{N}$ (f8),${}^{15}\mathrm{N}$ (f8),${}^{16}\mathrm{O}$ (f8),${}^{17}\mathrm{O}$ (f8),${}^{18}\mathrm{O}$ (f8),${}^{19}\mathrm{F}$ (f8),${}^{20}\mathrm{Ne}$ (f8),${}^{21}\mathrm{Ne}$ (f8),${}^{22}\mathrm{Ne}$ (f8),${}^{23}\mathrm{Na}$ (f8),${}^{24}\mathrm{Mg}$ (f8),${}^{25}\mathrm{Mg}$ (f8),${}^{26}\mathrm{Mg}$ (f8),${}^{27}\mathrm{Al}$ (f8),${}^{28}\mathrm{Si}$ (f8),${}^{29}\mathrm{Si}$ (f8),${}^{30}\mathrm{Si}$ (f8),${}^{31}\mathrm{P}$ (f8),${}^{32}\mathrm{S}$ (f8),${}^{33}\mathrm{S}$ (f8),${}^{34}\mathrm{S}$ (f8),${}^{36}\mathrm{S}$ (f8),${}^{35}\mathrm{Cl}$ (f8),${}^{37}\mathrm{Cl}$ (f8),${}^{36}\mathrm{Ar}$ (f8),${}^{38}\mathrm{Ar}$ (f8),${}^{40}\mathrm{Ar}$ (f8),${}^{39}\mathrm{K}$ (f8),${}^{40}\mathrm{K}$ (f8),${}^{41}\mathrm{K}$ (f8),${}^{40}\mathrm{Ca}$ (f8),${}^{42}\mathrm{Ca}$ (f8),${}^{43}\mathrm{Ca}$ (f8),${}^{44}\mathrm{Ca}$ (f8),${}^{46}\mathrm{Ca}$ (f8),${}^{48}\mathrm{Ca}$ (f8),${}^{45}\mathrm{Sc}$ (f8),${}^{46}\mathrm{Ti}$ (f8),${}^{47}\mathrm{Ti}$ (f8),${}^{48}\mathrm{Ti}$ (f8),${}^{49}\mathrm{Ti}$ (f8),${}^{50}\mathrm{Ti}$ (f8),${}^{50}\mathrm{V}$ (f8),${}^{51}\mathrm{V}$ (f8),${}^{50}\mathrm{Cr}$ (f8),${}^{52}\mathrm{Cr}$ (f8),${}^{53}\mathrm{Cr}$ (f8),${}^{54}\mathrm{Cr}$ (f8),${}^{55}\mathrm{Mn}$ (f8),${}^{54}\mathrm{Fe}$ (f8),${}^{56}\mathrm{Fe}$ (f8),${}^{57}\mathrm{Fe}$ (f8),${}^{58}\mathrm{Fe}$ (f8),${}^{59}\mathrm{Co}$ (f8),${}^{58}\mathrm{Ni}$ (f8),${}^{60}\mathrm{Ni}$ (f8),${}^{61}\mathrm{Ni}$ (f8),${}^{62}\mathrm{Ni}$ (f8),${}^{64}\mathrm{Ni}$ (f8),${}^{63}\mathrm{Cu}$ (f8),${}^{65}\mathrm{Cu}$ (f8),${}^{64}\mathrm{Zn}$ (f8),${}^{66}\mathrm{Zn}$ (f8),${}^{67}\mathrm{Zn}$ (f8),${}^{68}\mathrm{Zn}$ (f8),${}^{70}\mathrm{Zn}$ (f8),${}^{69}\mathrm{Ga}$ (f8),${}^{71}\mathrm{Ga}$ (f8),${}^{70}\mathrm{Ge}$ (f8),${}^{72}\mathrm{Ge}$ (f8),${}^{73}\mathrm{Ge}$ (f8),${}^{74}\mathrm{Ge}$ (f8),${}^{76}\mathrm{Ge}$ (f8),${}^{75}\mathrm{As}$ (f8),${}^{74}\mathrm{Se}$ (f8),${}^{76}\mathrm{Se}$ (f8),${}^{77}\mathrm{Se}$ (f8),${}^{78}\mathrm{Se}$ (f8),${}^{80}\mathrm{Se}$ (f8),${}^{82}\mathrm{Se}$ (f8),${}^{79}\mathrm{Br}$ (f8),${}^{81}\mathrm{Br}$ (f8),${}^{78}\mathrm{Kr}$ (f8),${}^{80}\mathrm{Kr}$ (f8),${}^{82}\mathrm{Kr}$ (f8),${}^{83}\mathrm{Kr}$ (f8),${}^{84}\mathrm{Kr}$ (f8),${}^{86}\mathrm{Kr}$ (f8),${}^{85}\mathrm{Rb}$ (f8),${}^{87}\mathrm{Rb}$ (f8),${}^{84}\mathrm{Sr}$ (f8),${}^{86}\mathrm{Sr}$ (f8),${}^{87}\mathrm{Sr}$ (f8),${}^{88}\mathrm{Sr}$ (f8),${}^{89}\mathrm{Y}$ (f8),${}^{90}\mathrm{Zr}$ (f8),${}^{91}\mathrm{Zr}$ (f8),${}^{92}\mathrm{Zr}$ (f8),${}^{94}\mathrm{Zr}$ (f8),${}^{96}\mathrm{Zr}$ (f8),${}^{93}\mathrm{Nb}$ (f8),${}^{92}\mathrm{Mo}$ (f8),${}^{94}\mathrm{Mo}$ (f8),${}^{95}\mathrm{Mo}$ (f8),${}^{96}\mathrm{Mo}$ (f8),${}^{97}\mathrm{Mo}$ (f8),${}^{98}\mathrm{Mo}$ (f8),${}^{100}\mathrm{Mo}$ (f8),${}^{96}\mathrm{Ru}$ (f8),${}^{98}\mathrm{Ru}$ (f8),${}^{99}\mathrm{Ru}$ (f8),${}^{100}\mathrm{Ru}$ (f8),${}^{101}\mathrm{Ru}$ (f8),${}^{102}\mathrm{Ru}$ (f8),${}^{104}\mathrm{Ru}$ (f8),${}^{103}\mathrm{Rh}$ (f8),${}^{102}\mathrm{Pd}$ (f8),${}^{104}\mathrm{Pd}$ (f8),${}^{105}\mathrm{Pd}$ (f8),${}^{106}\mathrm{Pd}$ (f8),${}^{108}\mathrm{Pd}$ (f8),${}^{110}\mathrm{Pd}$ (f8),${}^{107}\mathrm{Ag}$ (f8),${}^{109}\mathrm{Ag}$ (f8),${}^{106}\mathrm{Cd}$ (f8),${}^{108}\mathrm{Cd}$ (f8),${}^{110}\mathrm{Cd}$ (f8),${}^{111}\mathrm{Cd}$ (f8),${}^{112}\mathrm{Cd}$ (f8),${}^{113}\mathrm{Cd}$ (f8),${}^{114}\mathrm{Cd}$ (f8),${}^{116}\mathrm{Cd}$ (f8),${}^{113}\mathrm{In}$ (f8),${}^{115}\mathrm{In}$ (f8),${}^{112}\mathrm{Sn}$ (f8),${}^{114}\mathrm{Sn}$ (f8),${}^{115}\mathrm{Sn}$ (f8),${}^{116}\mathrm{Sn}$ (f8),${}^{117}\mathrm{Sn}$ (f8),${}^{118}\mathrm{Sn}$ (f8),${}^{119}\mathrm{Sn}$ (f8),${}^{120}\mathrm{Sn}$ (f8),${}^{122}\math

You can of course also create new reference values by passing a dict to the {func}`isopy.asrefval` function.

---
One special feature of reference values is that the {meth}`RefValDict.get()<isopy.core.RefValDict.get>` method can compute missing values for ratio and molecule key strings from the contents of the array, e.g.

In [33]:
isopy.asrefval('isotope.fraction').get('108pd/105pd') # Divides the value for 108Pd by 105Pd

1.1849529780564263

In [34]:
isopy.asrefval('isotope.fraction').get('(16O)(1H)') # Multiplies the value for 16O by the value for 1H

0.997465230567756

For more information on how these are computed see the documentation of {meth}`RefValDict.get()<isopy.core.RefValDict.get>`.

---
Reference values also work with array functions. Combining isopy arrays and reference values will return an array consisting of **only** of the keys in the array.

In [35]:
a = isopy.ones(1, '101ru 105pd 111cd')
a * isopy.asrefval('isotope.fraction')

(row),${}^{101}\mathrm{Ru}$ (f8),${}^{105}\mathrm{Pd}$ (f8),${}^{111}\mathrm{Cd}$ (f8)
0,0.17060,0.22330,0.12804


In [36]:
a = isopy.ones(1, '101ru/99ru 108pd/105pd 111cd/116cd') # Calculates the ratios automatically
a * isopy.asrefval('isotope.fraction')

(row),$\cfrac{{}^{101}\mathrm{Ru}}{{}^{99}\mathrm{Ru}}$ (f8),$\cfrac{{}^{108}\mathrm{Pd}}{{}^{105}\mathrm{Pd}}$ (f8),$\cfrac{{}^{111}\mathrm{Cd}}{{}^{116}\mathrm{Cd}}$ (f8)
0,1.33721,1.18495,1.70697


You can also use numpy functions on reference values. It might be considerably faster to use the function in the isopy namespace together with the `keys` argument to avoid computing the result for unused keys.

In [37]:
isopy.log(isopy.asrefval('isotope.fraction'), keys='101ru 105pd 111cd')

(row),${}^{101}\mathrm{Ru}$ (f8),${}^{105}\mathrm{Pd}$ (f8),${}^{111}\mathrm{Cd}$ (f8)
None,-1.76843,-1.49924,-2.05541


---
You can convert reference values, or a subsection thereof, to an array using the {meth}`RefValDict.to_array()<isopy.core.RefValDict.to_array>`function. The first argument is the keys the array should contain.

In [38]:
isopy.asrefval('isotope.fraction').to_array('101ru 105pd 111cd')

(row),${}^{101}\mathrm{Ru}$ (f8),${}^{105}\mathrm{Pd}$ (f8),${}^{111}\mathrm{Cd}$ (f8)
None,0.17060,0.22330,0.12804
